# Capítulo 7: Vetorização de Joins

## Introdução

Hash Joins são uma das operações mais importantes em SQL. O DuckDB otimiza joins usando **prefetching** para esconder latência de memória e **processamento vetorizado** em todas as fases.

### Objetivos:
1. Entender as fases Build e Probe de Hash Join
2. Implementar prefetching para esconder latência
3. Comparar Nested Loop vs Hash Join
4. Otimizar para diferentes tamanhos de tabela
5. Entender Bloom Filters para joins

In [ ]:
!pip install duckdb pandas numpy matplotlib -q

In [ ]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from collections import defaultdict

plt.rcParams['figure.figsize'] = (12, 6)

## 7.1 Hash Join: Build vs Probe

```
┌──────────────────────────────────────────────────────────────┐
│                     HASH JOIN                                │
├──────────────────────────────────────────────────────────────┤
│                                                              │
│  FASE BUILD (tabela menor):                                  │
│    1. Para cada linha da tabela build                        │
│    2. Calcular hash da chave de join                         │
│    3. Inserir (chave, valor) na hash table                   │
│                                                              │
│  ┌─────────────┐                                             │
│  │ Build Table │  →  ┌─────────────────────┐                 │
│  │   (menor)   │     │    Hash Table       │                 │
│  └─────────────┘     │  bucket[0]: [...]   │                 │
│                      │  bucket[1]: [...]   │                 │
│                      │  ...                │                 │
│                      └─────────────────────┘                 │
│                                                              │
│  FASE PROBE (tabela maior):                                  │
│    1. Para cada linha da tabela probe                        │
│    2. Calcular hash da chave de join                         │
│    3. Buscar matches na hash table                           │
│    4. Emitir resultado para cada match                       │
│                                                              │
│  ┌─────────────┐     ┌─────────────────────┐                 │
│  │ Probe Table │  →  │    Hash Table       │  →  Resultado   │
│  │   (maior)   │     │  (lookup)           │                 │
│  └─────────────┘     └─────────────────────┘                 │
│                                                              │
└──────────────────────────────────────────────────────────────┘

Complexidade: O(n + m) vs O(n × m) do Nested Loop!
```

In [ ]:
# Implementação básica de Hash Join (linha por linha)
def hash_join_basic(build_keys, build_values, probe_keys, probe_values):
    """Hash Join básico - processa uma linha por vez"""
    
    # FASE BUILD: construir hash table
    hash_table = defaultdict(list)
    for i in range(len(build_keys)):
        key = build_keys[i]
        hash_table[key].append(build_values[i])
    
    # FASE PROBE: buscar matches
    result_probe = []
    result_build = []
    
    for i in range(len(probe_keys)):
        key = probe_keys[i]
        if key in hash_table:
            for build_val in hash_table[key]:
                result_probe.append(probe_values[i])
                result_build.append(build_val)
    
    return result_probe, result_build

# Teste
np.random.seed(42)
build_keys = np.random.randint(0, 1000, 10000)   # 10K linhas
build_values = np.arange(10000)
probe_keys = np.random.randint(0, 1000, 100000)  # 100K linhas
probe_values = np.arange(100000)

start = time.perf_counter()
r1, r2 = hash_join_basic(build_keys, build_values, probe_keys, probe_values)
basic_time = time.perf_counter() - start

print(f"Hash Join básico: {basic_time*1000:.2f} ms")
print(f"Matches encontrados: {len(r1):,}")

In [ ]:
# Nested Loop Join para comparação
def nested_loop_join(build_keys, build_values, probe_keys, probe_values, limit=10000):
    """Nested Loop Join - O(n × m), muito lento!"""
    result_probe = []
    result_build = []
    
    # Limitar para não demorar demais
    for i in range(min(len(probe_keys), limit)):
        for j in range(len(build_keys)):
            if probe_keys[i] == build_keys[j]:
                result_probe.append(probe_values[i])
                result_build.append(build_values[j])
    
    return result_probe, result_build

# Benchmark com subset menor
limit = 1000
start = time.perf_counter()
r1_nl, r2_nl = nested_loop_join(build_keys, build_values, probe_keys, probe_values, limit)
nl_time = time.perf_counter() - start

print(f"Nested Loop ({limit} probe rows): {nl_time*1000:.2f} ms")
print(f"Estimativa para 100K rows: {nl_time * 100:.0f} ms")
print(f"Hash Join é ~{(nl_time * 100) / basic_time:.0f}x mais rápido!")

## 7.2 Prefetching: Escondendo Latência

O problema com hash joins é que os acessos à hash table são **aleatórios**:

```
SEM PREFETCH:
  Probe key 0 → hash → lookup → WAIT (cache miss) → resultado
  Probe key 1 → hash → lookup → WAIT (cache miss) → resultado
  ...
  CPU fica parada esperando memória!

COM PREFETCH:
  1. Calcular hashes para keys 0-7
  2. Emitir prefetch para posições 0-7 da hash table
  3. Enquanto RAM busca dados:
     - Processar resultados do batch anterior!
  4. Repetir
  
  CPU nunca fica ociosa!
```

In [ ]:
# Simulação de Hash Join com prefetching (conceitual)
def hash_join_with_prefetch_simulation(build_keys, build_values, probe_keys, probe_values, batch_size=64):
    """
    Hash Join simulando prefetching.
    Em C++, usaríamos _mm_prefetch para instruções reais.
    """
    
    # FASE BUILD
    hash_table = defaultdict(list)
    for i in range(len(build_keys)):
        hash_table[build_keys[i]].append(build_values[i])
    
    # FASE PROBE com batching (simula prefetch)
    result_probe = []
    result_build = []
    
    n_probe = len(probe_keys)
    
    for batch_start in range(0, n_probe, batch_size):
        batch_end = min(batch_start + batch_size, n_probe)
        
        # 1. Calcular hashes do batch (vetorizado)
        batch_keys = probe_keys[batch_start:batch_end]
        batch_values = probe_values[batch_start:batch_end]
        
        # 2. "Prefetch" - em C++ seria _mm_prefetch
        # Aqui apenas organizamos o trabalho em batches
        
        # 3. Processar batch
        for i, key in enumerate(batch_keys):
            if key in hash_table:
                for build_val in hash_table[key]:
                    result_probe.append(batch_values[i])
                    result_build.append(build_val)
    
    return result_probe, result_build

# Benchmark
start = time.perf_counter()
r1_pf, r2_pf = hash_join_with_prefetch_simulation(
    build_keys, build_values, probe_keys, probe_values, batch_size=64
)
prefetch_time = time.perf_counter() - start

print(f"Hash Join com prefetch simulado: {prefetch_time*1000:.2f} ms")
print(f"Matches: {len(r1_pf):,}")

## 7.3 Hash Join Vetorizado

Processa múltiplas chaves de uma vez usando NumPy.

In [ ]:
def hash_join_vectorized(build_keys, build_values, probe_keys, probe_values):
    """
    Hash Join usando operações vetorizadas NumPy.
    Mais próximo de como o DuckDB implementa.
    """
    
    # Build: criar índice de posições para cada chave
    # Usando sorting para agrupar chaves iguais
    build_sort_idx = np.argsort(build_keys)
    build_keys_sorted = build_keys[build_sort_idx]
    build_values_sorted = build_values[build_sort_idx]
    
    # Encontrar onde cada chave única começa
    unique_build, build_starts = np.unique(build_keys_sorted, return_index=True)
    build_ends = np.append(build_starts[1:], len(build_keys_sorted))
    
    # Criar dicionário de ranges
    key_to_range = {k: (build_starts[i], build_ends[i]) for i, k in enumerate(unique_build)}
    
    # Probe: encontrar quais probe keys existem no build
    probe_in_build = np.isin(probe_keys, unique_build)
    matching_probe_idx = np.where(probe_in_build)[0]
    
    # Coletar resultados
    result_probe = []
    result_build = []
    
    for idx in matching_probe_idx:
        key = probe_keys[idx]
        start, end = key_to_range[key]
        
        for build_idx in range(start, end):
            result_probe.append(probe_values[idx])
            result_build.append(build_values_sorted[build_idx])
    
    return np.array(result_probe), np.array(result_build)

# Benchmark
start = time.perf_counter()
r1_vec, r2_vec = hash_join_vectorized(build_keys, build_values, probe_keys, probe_values)
vec_time = time.perf_counter() - start

print(f"Hash Join vetorizado: {vec_time*1000:.2f} ms")
print(f"Matches: {len(r1_vec):,}")
print(f"Speedup vs básico: {basic_time/vec_time:.1f}x")

## 7.4 Bloom Filters

Bloom Filters permitem descartar rapidamente chaves que **certamente não têm match**:

```
BLOOM FILTER:
  - Estrutura probabilística
  - Responde: "Definitivamente não está" ou "Talvez esteja"
  - Muito compacto (bits em vez de valores)

USO EM JOINS:
  1. Durante BUILD: inserir todas as chaves no Bloom Filter
  2. Durante PROBE: primeiro checar Bloom Filter
     - Se "não está": pular (evita lookup na hash table)
     - Se "talvez": fazer lookup normal

Benefício: Evita lookups desnecessários na hash table!
```

In [ ]:
class SimpleBloomFilter:
    """Implementação simples de Bloom Filter"""
    
    def __init__(self, size: int = 10000, num_hashes: int = 3):
        self.size = size
        self.num_hashes = num_hashes
        self.bits = np.zeros(size, dtype=bool)
    
    def _hashes(self, key):
        """Gera múltiplos hashes para uma chave"""
        hashes = []
        for i in range(self.num_hashes):
            h = hash((key, i)) % self.size
            hashes.append(h)
        return hashes
    
    def add(self, key):
        """Adiciona chave ao filtro"""
        for h in self._hashes(key):
            self.bits[h] = True
    
    def might_contain(self, key) -> bool:
        """Retorna True se a chave PODE estar presente"""
        return all(self.bits[h] for h in self._hashes(key))
    
    def add_batch(self, keys):
        """Adiciona múltiplas chaves (vetorizado quando possível)"""
        for key in keys:
            self.add(key)

# Demonstração
bloom = SimpleBloomFilter(size=50000, num_hashes=3)

# Adicionar chaves do build
unique_build_keys = np.unique(build_keys)
bloom.add_batch(unique_build_keys)

# Testar algumas chaves
test_keys = [0, 1, 999, 1500, 2000]  # 0-999 devem existir, 1500+ não
for k in test_keys:
    result = bloom.might_contain(k)
    actually_exists = k in unique_build_keys
    print(f"Key {k}: Bloom diz '{result}', realmente existe: {actually_exists}")

In [ ]:
def hash_join_with_bloom(build_keys, build_values, probe_keys, probe_values):
    """
    Hash Join usando Bloom Filter para filtrar probes.
    """
    # Criar Bloom Filter durante BUILD
    bloom = SimpleBloomFilter(size=len(build_keys) * 10, num_hashes=3)
    
    hash_table = defaultdict(list)
    for i in range(len(build_keys)):
        key = build_keys[i]
        hash_table[key].append(build_values[i])
        bloom.add(key)
    
    # PROBE com Bloom Filter
    result_probe = []
    result_build = []
    bloom_filtered = 0
    
    for i in range(len(probe_keys)):
        key = probe_keys[i]
        
        # Primeiro: checar Bloom Filter
        if not bloom.might_contain(key):
            bloom_filtered += 1
            continue  # Definitivamente não tem match!
        
        # Talvez tenha match: fazer lookup normal
        if key in hash_table:
            for build_val in hash_table[key]:
                result_probe.append(probe_values[i])
                result_build.append(build_val)
    
    return result_probe, result_build, bloom_filtered

# Teste com probe keys que têm muitos non-matches
probe_keys_sparse = np.random.randint(0, 5000, 100000)  # Muitos não existirão
probe_values_sparse = np.arange(100000)

start = time.perf_counter()
r1_bloom, r2_bloom, filtered = hash_join_with_bloom(
    build_keys, build_values, probe_keys_sparse, probe_values_sparse
)
bloom_time = time.perf_counter() - start

# Sem Bloom
start = time.perf_counter()
r1_no_bloom, r2_no_bloom = hash_join_basic(
    build_keys, build_values, probe_keys_sparse, probe_values_sparse
)
no_bloom_time = time.perf_counter() - start

print(f"Sem Bloom Filter: {no_bloom_time*1000:.2f} ms")
print(f"Com Bloom Filter: {bloom_time*1000:.2f} ms")
print(f"Probes filtrados pelo Bloom: {filtered:,} ({100*filtered/len(probe_keys_sparse):.1f}%)")
print(f"Speedup: {no_bloom_time/bloom_time:.2f}x")

## 7.5 Benchmark: Impacto do Tamanho das Tabelas

In [ ]:
# Variar tamanho da tabela build
probe_size = 100000
probe_keys_fixed = np.random.randint(0, 10000, probe_size)
probe_values_fixed = np.arange(probe_size)

build_sizes = [100, 500, 1000, 5000, 10000, 50000, 100000]
results = []

for bs in build_sizes:
    build_keys_test = np.random.randint(0, 10000, bs)
    build_values_test = np.arange(bs)
    
    times = []
    for _ in range(3):
        start = time.perf_counter()
        _ = hash_join_basic(build_keys_test, build_values_test, 
                           probe_keys_fixed, probe_values_fixed)
        times.append(time.perf_counter() - start)
    
    avg_time = np.mean(times) * 1000
    results.append({'build_size': bs, 'time_ms': avg_time})
    print(f"Build size {bs:>6}: {avg_time:6.2f} ms")

In [ ]:
# Visualização
df_results = pd.DataFrame(results)

plt.figure(figsize=(10, 5))
plt.plot(df_results['build_size'], df_results['time_ms'], marker='o', linewidth=2)
plt.xlabel('Tamanho da Tabela Build')
plt.ylabel('Tempo (ms)')
plt.title('Tempo de Hash Join vs Tamanho da Tabela Build\n(Probe fixo em 100K linhas)')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.show()

## 7.6 DuckDB: Joins em Ação

In [ ]:
con = duckdb.connect(':memory:')

# Criar tabelas
con.execute("""
    CREATE TABLE produtos AS
    SELECT 
        i AS produto_id,
        'Produto_' || LPAD(i::VARCHAR, 5, '0') AS nome,
        random() * 100 AS preco,
        (random() * 10)::INTEGER AS categoria_id
    FROM generate_series(1, 10000) AS t(i)
""")

con.execute("""
    CREATE TABLE vendas AS
    SELECT 
        i AS venda_id,
        (random() * 10000)::INTEGER + 1 AS produto_id,
        random() * 100 AS quantidade,
        DATE '2024-01-01' + (random() * 365)::INTEGER AS data_venda
    FROM generate_series(1, 10000000) AS t(i)
""")

con.execute("""
    CREATE TABLE categorias AS
    SELECT 
        i AS categoria_id,
        'Categoria_' || i AS nome_categoria
    FROM generate_series(0, 10) AS t(i)
""")

print("Tabelas criadas:")
print("  - produtos: 10K linhas")
print("  - vendas: 10M linhas")
print("  - categorias: 11 linhas")

In [ ]:
# Benchmark de diferentes tipos de joins
queries = {
    'INNER JOIN simples': '''
        SELECT COUNT(*) 
        FROM vendas v 
        INNER JOIN produtos p ON v.produto_id = p.produto_id
    ''',
    'INNER JOIN com projeção': '''
        SELECT v.venda_id, p.nome, v.quantidade
        FROM vendas v 
        INNER JOIN produtos p ON v.produto_id = p.produto_id
        LIMIT 1000000
    ''',
    'JOIN + Agregação': '''
        SELECT p.nome, SUM(v.quantidade * p.preco) as total
        FROM vendas v
        INNER JOIN produtos p ON v.produto_id = p.produto_id
        GROUP BY p.nome
    ''',
    'LEFT JOIN': '''
        SELECT COUNT(*)
        FROM produtos p
        LEFT JOIN vendas v ON p.produto_id = v.produto_id
    ''',
    'Multi-table JOIN': '''
        SELECT c.nome_categoria, COUNT(*) as total_vendas
        FROM vendas v
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN categorias c ON p.categoria_id = c.categoria_id
        GROUP BY c.nome_categoria
    ''',
    'JOIN com filtro': '''
        SELECT COUNT(*)
        FROM vendas v
        JOIN produtos p ON v.produto_id = p.produto_id
        WHERE p.preco > 50 AND v.quantidade > 50
    ''',
}

print("Performance de Joins DuckDB:\n")
join_results = []
for name, query in queries.items():
    times = []
    for _ in range(5):
        start = time.perf_counter()
        con.execute(query).fetchall()
        times.append(time.perf_counter() - start)
    
    avg_time = np.mean(times) * 1000
    join_results.append({'query': name, 'time_ms': avg_time})
    print(f"{name:30} {avg_time:8.2f} ms")

In [ ]:
# Visualização
df_joins = pd.DataFrame(join_results)

plt.figure(figsize=(12, 6))
bars = plt.barh(df_joins['query'], df_joins['time_ms'], color='steelblue')
plt.xlabel('Tempo (ms)')
plt.title('Performance de Diferentes Joins no DuckDB')
plt.grid(True, alpha=0.3, axis='x')

for bar, val in zip(bars, df_joins['time_ms']):
    plt.text(val + 5, bar.get_y() + bar.get_height()/2, f'{val:.1f}ms', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Ver plano de execução
print("=== EXPLAIN ===")
plan = con.execute("""
    EXPLAIN
    SELECT p.nome, SUM(v.quantidade * p.preco) as total
    FROM vendas v
    JOIN produtos p ON v.produto_id = p.produto_id
    GROUP BY p.nome
""").df()
print(plan.to_string())

In [ ]:
# Explain analyze para ver estatísticas
print("\n=== EXPLAIN ANALYZE ===")
plan = con.execute("""
    EXPLAIN ANALYZE
    SELECT COUNT(*) 
    FROM vendas v 
    INNER JOIN produtos p ON v.produto_id = p.produto_id
""").df()
print(plan.to_string())

## 7.7 Comparação: DuckDB vs Pandas

In [ ]:
# Carregar dados para Pandas (subset para não demorar muito)
df_vendas = con.execute("SELECT produto_id, quantidade FROM vendas").df()
df_produtos = con.execute("SELECT produto_id, nome, preco FROM produtos").df()

print(f"Vendas: {len(df_vendas):,} linhas")
print(f"Produtos: {len(df_produtos):,} linhas")

# Benchmark: JOIN simples
# DuckDB
times_duck = []
for _ in range(5):
    start = time.perf_counter()
    con.execute("""
        SELECT COUNT(*) 
        FROM vendas v 
        JOIN produtos p ON v.produto_id = p.produto_id
    """).fetchall()
    times_duck.append(time.perf_counter() - start)

# Pandas
times_pandas = []
for _ in range(5):
    start = time.perf_counter()
    merged = df_vendas.merge(df_produtos, on='produto_id')
    count = len(merged)
    times_pandas.append(time.perf_counter() - start)

print(f"\nINNER JOIN (10M × 10K):")
print(f"DuckDB: {np.mean(times_duck)*1000:.2f} ms")
print(f"Pandas: {np.mean(times_pandas)*1000:.2f} ms")
print(f"Speedup: {np.mean(times_pandas)/np.mean(times_duck):.1f}x")

## 7.8 Resumo

| Otimização | Descrição | Benefício |
|------------|-----------|----------|
| **Build menor** | Tabela menor vira hash table | Cabe no cache |
| **Prefetching** | Emite loads antes de precisar | Esconde latência RAM |
| **Probe vetorizado** | Processa múltiplas chaves por vez | SIMD + cache |
| **Bloom filters** | Filtra probes que não têm match | Evita lookups |
| **Particionamento** | Divide hash table por threads | Paralelismo |

### Pontos Chave:

1. **Hash Join O(n+m)**: Muito melhor que Nested Loop O(n×m)
2. **Build a tabela menor**: Hash table menor = mais cache hits
3. **Prefetching**: CPU nunca fica ociosa esperando memória
4. **Bloom Filter**: Descarta non-matches rapidamente
5. **Vetorização**: Processa batches de chaves

In [ ]:
con.close()